<a href="https://colab.research.google.com/github/hojunking/carbon_reduction_project/blob/main/model01_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import

In [1]:
import random
import pandas as pd
import numpy as np
import os
import cv2
import time, datetime

from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GroupKFold, StratifiedKFold
from sklearn import metrics
from sklearn.metrics import roc_auc_score, log_loss, f1_score, confusion_matrix, classification_report

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.distributed import DistributedSampler
import torchvision.models as models
from torch.cuda.amp import autocast, GradScaler

from tqdm.auto import tqdm
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2



import warnings
warnings.filterwarnings(action='ignore') 

In [2]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device

device(type='cuda')

## Hyperparameter Setting

In [3]:
CFG = {
    'fold_num': 3,
    'seed': 42,
    'model': 'resnet50',
    'img_size': 260,
    'epochs': 200,
    'train_bs':128,
    'valid_bs':64,
    'T_0': 10,
    'lr': 1e-4,
    'min_lr': 1e-6,
    'num_workers': 0,
    'accum_iter': 2, # suppoprt to do batch accumulation for backprop with effectively larger batch size
    'verbose_step': 1,
    'patience' : 5,
    'device': 'cuda:0',
    'freezing': False,
    'model_path': '/content/drive/MyDrive/탄소저감프로젝트/models'
}

## Fixed RandomSeed

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG['seed']) # Seed 고정

## Data Pre-processing

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
main_path = '/content/drive/MyDrive/탄소저감프로젝트/data/'
label_list = ["10kwalking","public_vehicle",'stair','pet','outlet','else', 'can', 'box', 'milk', 'cup']

total_train_img_paths = []
total_train_img_labels = []
total_test_img_paths = []
total_test_img_labels = []

for label in label_list: ## 각 레이블 돌기
  print(f'label: {label}')
  img_paths = [] 
  img_labels = []

  # default ratio
  train_ratio = 1000
  test_ratio = 400

  dir_path = main_path + label ## 레이블 폴더 경로
  count = 0
  for folder, subfolders, filenames in os.walk(dir_path): ## 폴더 내 모든 파일 탐색
    
    for img in filenames: ## 각 파일 경로, 레이블 저장
      count +=1
      if count > train_ratio + test_ratio:
        break
      
      img_paths.append(folder+'/'+img)
      img_labels.append(label)

  print(f'img_paths len : {len(img_paths)}\n')

  if label == '10kwalking': ## 10walking 데이터 비율 설정하기 (데이터수: 2494)
    train_ratio = 1000
    test_ratio = 400
  elif label == 'public_vehicle': ## 10walking 데이터 비율 설정하기 (데이터수: 2494)
    train_ratio = 1000
    test_ratio = 400

  total_train_img_paths.extend(img_paths[:train_ratio])
  total_train_img_labels.extend(img_labels[:train_ratio])
  
  total_test_img_paths.extend(img_paths[-test_ratio:])
  total_test_img_labels.extend(img_labels[-test_ratio:])

print('Train_Images: ',len(total_train_img_paths))
print("Train_Images_labels:", len(total_train_img_labels))
print('Test_Images: ',len(total_test_img_paths))
print("Test_Images_labels:", len(total_test_img_labels))

label: 10kwalking
img_paths len : 1400

label: public_vehicle
img_paths len : 1245

label: stair
img_paths len : 1400

label: pet
img_paths len : 1400

label: outlet
img_paths len : 1400

label: else
img_paths len : 1400

label: can
img_paths len : 1400

label: box
img_paths len : 1400

label: milk
img_paths len : 1400

label: cup
img_paths len : 1068

Train_Images:  10000
Train_Images_labels: 10000
Test_Images:  4000
Test_Images_labels: 4000


In [ ]:
total_train_img_labels

In [ ]:
## Pandas 데이터프레임 만들기
trn_df = pd.DataFrame(total_train_img_paths, columns=['image_id'])
trn_df['dir'] = trn_df['image_id'].apply(lambda x: os.path.dirname(x))
trn_df['image_id'] = trn_df['image_id'].apply(lambda x: os.path.basename(x))
trn_df['label'] = total_train_img_labels
train = trn_df
train

,image_id,dir,label
0,캐시워크 만보_352.jpg,/content/drive/MyDrive/탄소저감프로젝트/dat...,10kwalking
1,캐시워크 만보_354.jpg,/content/drive/MyDrive/탄소저감프로젝트/dat...,10kwalking
2,캐시워크 만보_355.jpg,/content/drive/MyDrive/탄소저감프로젝트/dat...,10kwalking
3,캐시워크 만보_353.jpg,/content/drive/MyDrive/탄소저감프로젝트/dat...,10kwalking
4,캐시워크 만보_356.jpg,/content/drive/MyDrive/탄소저감프로젝트/dat...,10kwalking
...,...,...,...
9995,양치컵_구글_313.jpg,/content/drive/MyDrive/탄소저감프로젝트/dat...,cup
9996,양치컵_구글_314.jpg,/content/drive/MyDrive/탄소저감프로젝트/dat...,cup
9997,양치컵_구글_315.jpg,/content/drive/MyDrive/탄소저감프로젝트/dat...,cup
9998,양치컵_구글_316.jpg,/content/drive/MyDrive/탄소저감프로젝트/dat...,cup


In [ ]:
# Label Encoding
le = preprocessing.LabelEncoder()
train['label'] = le.fit_transform(train['label'].values)

In [ ]:
def get_img(path, train_arg=False, sub_path=None):
    try:
        im_bgr = cv2.imread(path)
        im_rgb = im_bgr[:, :, ::-1]
        past_path = path
    except: ## 이미지 에러 발생 시 백지로 대체
        im_bgr = cv2.imread('/content/drive/MyDrive/탄소저감프로젝트/temp_img.jpg')
        im_rgb = im_bgr[:, :, ::-1]
    if train_arg:
        dimg = cv2.fastNlMeansDenoisingColored(im_rgb, None, 10,10,7,21)
        return dimg
    else:
        return im_rgb

## CustomDataset

In [ ]:
class ColonDataset(Dataset):
    def __init__(self, df, data_root, train_arg=False,
                 transform=None,
                 output_label=True
                ):
        
        super().__init__()
        self.df = df.reset_index(drop=True).copy()
        self.train_arg = train_arg
        self.transform = transform
        self.data_root = data_root
        self.output_label = output_label
        
        if output_label == True:
            self.labels = self.df['label'].values
            
    def __len__(self):
        return self.df.shape[0]
    
    def __getitem__(self, index: int):
        
        # get labels
        if self.output_label:
            target = self.labels[index]
        
        img  = get_img("{}/{}".format(self.data_root[index], self.df.loc[index]['image_id']), train_arg=self.train_arg)

        if self.transform:
            transformed = self.transform(image=img)
            img = transformed['image']
        
                
        if self.output_label == True:
            return img, target
        else:
            return img

## Data Load

In [ ]:
transform_train = A.Compose(
    [
        A.Resize(height = CFG['img_size']+100, width = CFG['img_size']+100),
        A.CenterCrop(always_apply=True, p=0.7, height=CFG['img_size'], width=CFG['img_size']),
        A.RandomBrightnessContrast(always_apply=True, p=0.8, brightness_limit=(0.00, 0.00), contrast_limit=(0.2, 0.2), brightness_by_max=False),
        A.SafeRotate(always_apply=True, p=0.5, limit=(-20, 20), interpolation=2, border_mode=0, value=(0, 0, 0), mask_value=None),
        A.HorizontalFlip(always_apply=False, p=0.5),
        A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
        A.pytorch.transforms.ToTensorV2()
        ])

transform_test = A.Compose(
    [
        A.Resize(height = CFG['img_size'], width = CFG['img_size']),
        A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
        A.pytorch.transforms.ToTensorV2()
        ])

In [ ]:
def prepare_dataloader(df, trn_idx, val_idx, data_root=train.dir.values):
    
    train_ = df.loc[trn_idx,:].reset_index(drop=True)
    valid_ = df.loc[val_idx,:].reset_index(drop=True)
    train_data_root = data_root[trn_idx]
    valid_data_root = data_root[val_idx]
    
        
    train_ds = ColonDataset(train_,
                            train_data_root,
                            transform=transform_train,
                            output_label=True)
    valid_ds = ColonDataset(valid_,
                            valid_data_root,
                            transform=transform_train,
                            output_label=True)
    

    train_loader = torch.utils.data.DataLoader(
        train_ds,
        batch_size=CFG['train_bs'],
        pin_memory=True,
        drop_last=False,
        shuffle=False,        
        num_workers=CFG['num_workers']
        #sampler=DistributedSampler(train_ds)
    )
    val_loader = torch.utils.data.DataLoader(
        valid_ds, 
        batch_size=CFG['valid_bs'],
        num_workers=CFG['num_workers'],
        shuffle=False,
        pin_memory=True,
    )
    return train_loader, val_loader

In [ ]:
class EarlyStopping:
    def __init__(self, patience=10, verbose=False, delta=0):
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta

    def __call__(self, score):
        if self.best_score is None:
            self.best_score = score
        elif score <= self.best_score + self.delta:
            self.counter += 1
            print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            print(f'Best F1 score from now: {self.best_score}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.counter = 0
        
        return self.early_stop

## Model Define

In [ ]:
class BaseModel(nn.Module):
    def __init__(self, model_arch, num_classes=len(le.classes_),pretrained=True):
        super(BaseModel, self).__init__()
        self.backbone = models.resnet50(pretrained=pretrained)
        self.classifier = nn.Linear(1000, num_classes)
        
    def forward(self, x):
        x = self.backbone(x)
        x = self.classifier(x)
        return x

## Train

In [ ]:
def train_one_epoch(epoch, model, loss_fn, optimizer, train_loader, device, scheduler=None, schd_batch_update=False):
    model.train()

    t = time.time()
    running_loss = None
    loss_sum = 0
    image_preds_all = []
    image_targets_all = []
    acc_list = []
    
    pbar = tqdm(enumerate(train_loader), total=len(train_loader))
    for step, (imgs, image_labels) in pbar:
        imgs = imgs.to(device).float()
        image_labels = image_labels.to(device).long()
        with autocast():
            image_preds = model(imgs)   #output = model(input)

            loss = loss_fn(image_preds, image_labels)
            loss_sum+=loss.detach()
            
            scaler.scale(loss).backward()
            if running_loss is None:
                running_loss = loss.item()
            else:
                running_loss = running_loss * .99 + loss.item() * .01

            if ((step + 1) %  CFG['accum_iter'] == 0) or ((step + 1) == len(train_loader)):

                scaler.step(optimizer)
                scaler.update()
                optimizer.zero_grad() 
                
                if scheduler is not None and schd_batch_update:
                    scheduler.step()
            if ((step + 1) % CFG['verbose_step'] == 0) or ((step + 1) == len(train_loader)):
                description = f'epoch {epoch} loss: {running_loss:.4f}'
                
                pbar.set_description(description)
            
        image_preds_all += [torch.argmax(image_preds, 1).detach().cpu().numpy()]
        image_targets_all += [image_labels.detach().cpu().numpy()]
    
    image_preds_all = np.concatenate(image_preds_all)
    image_targets_all = np.concatenate(image_targets_all)
    matrix = confusion_matrix(image_targets_all,image_preds_all)
    epoch_f1 = f1_score(image_targets_all, 
                        image_preds_all, 
                        average='macro')
    
    accuracy = (image_preds_all==image_targets_all).mean()
    trn_loss = loss_sum/len(train_loader)
    
    if scheduler is not None and not schd_batch_update:
        scheduler.step()
    
    return image_preds_all, accuracy, trn_loss, matrix, epoch_f1

def valid_one_epoch(epoch, model, loss_fn, val_loader, device, scheduler=None, schd_loss_update=False):
    model.eval()

    t = time.time()
    loss_sum = 0
    sample_num = 0
    avg_loss = 0
    image_preds_all = []
    image_targets_all = []
    
    acc_list = []
    
    pbar = tqdm(enumerate(val_loader), total=len(val_loader))
    for step, (imgs, image_labels) in pbar:
        imgs = imgs.to(device).float()
        image_labels = image_labels.to(device).long()
        
        image_preds = model(imgs)   #output = model(input)
        image_preds_all += [torch.argmax(image_preds, 1).detach().cpu().numpy()]
        image_targets_all += [image_labels.detach().cpu().numpy()]
        
        loss = loss_fn(image_preds, image_labels)
        avg_loss += loss.item()
        loss_sum += loss.item()*image_labels.shape[0]
        sample_num += image_labels.shape[0]
        
        if ((step + 1) % CFG['verbose_step'] == 0) or ((step + 1) == len(val_loader)):
            description = f'epoch {epoch} loss: {loss_sum/sample_num:.4f}'
            pbar.set_description(description)
    
    image_preds_all = np.concatenate(image_preds_all)
    image_targets_all = np.concatenate(image_targets_all)
    matrix = confusion_matrix(image_targets_all,image_preds_all)
    
    epoch_f1 = f1_score(image_targets_all, 
                        image_preds_all, 
                        average='macro')
    
    acc = (image_preds_all==image_targets_all).mean()
    val_loss = avg_loss/len(val_loader)
    
    if scheduler is not None:
        if schd_loss_update:
            scheduler.step(loss_sum/sample_num)
        else:
            scheduler.step()
            
    return image_preds_all, acc, val_loss, matrix, epoch_f1

In [ ]:
model_dir = '{}'.format(CFG['model'])
train_dir = train.dir.values

if __name__ == '__main__':
    
    seed_everything(CFG['seed'])
    
    if not os.path.isdir(model_dir):
        os.makedirs(model_dir)
    
    folds = StratifiedKFold(n_splits=CFG['fold_num'], shuffle=True, random_state=CFG['seed']).split(np.arange(train.shape[0]), train.label.values)
    
    best_fold = 0
    model_name = CFG['model']
    print(f'Model: {model_name}')
    for fold, (trn_idx, val_idx) in enumerate(folds):
    
        print(f'Training start with fold: {fold} epoch: {CFG["epochs"]} \n')

        early_stopping = EarlyStopping(patience=CFG["patience"], verbose=True)
        train_loader, val_loader = prepare_dataloader(train, trn_idx, val_idx, data_root=train_dir)

        device = torch.device(CFG['device'])
        model = BaseModel(CFG['model'], train.label.nunique(), pretrained=True)
        #model.freezing(freeze = CFG['freezing'])

        for name, param in model.named_parameters():
            if CFG['freezing'] == True and param.requires_grad:
                print(f'Freezing process: Not freezed : {name}')
        if CFG['freezing'] == False:
            print('All training process')
        
        model.to(device)
        scaler = GradScaler()   
        optimizer = torch.optim.Adam(model.parameters(), lr=CFG['lr'])
        scheduler = torch.optim.lr_scheduler.StepLR(optimizer, gamma=0.5, step_size=25)

        loss_tr = nn.CrossEntropyLoss().to(device) #MyCrossEntropyLoss().to(device)
        loss_fn = nn.CrossEntropyLoss().to(device)


        train_acc_list = []
        train_matrix_list = []
        train_f1_list = []
        valid_acc_list = []
        valid_matrix_list = []
        valid_f1_list = []
        best_f1 =0.0
        
        start = time.time()
        for epoch in range(CFG['epochs']):
            print(f'Fold: {fold}')
            print('Epoch {}/{}'.format(epoch, CFG['epochs'] - 1))
            
            train_preds_all, train_acc, train_loss, train_matrix, train_f1 = train_one_epoch(epoch, model, loss_tr,
                                                                        optimizer, train_loader, device, scheduler=scheduler, schd_batch_update=False)

            with torch.no_grad():
                valid_preds_all, valid_acc, valid_loss, valid_matrix, valid_f1= valid_one_epoch(epoch, model, loss_fn,
                                                                        val_loader, device, scheduler=None, schd_loss_update=False)
                
            train_acc_list.append(train_acc)
            train_matrix_list.append(train_matrix)
            train_f1_list.append(train_f1)
            
            valid_acc_list.append(valid_acc)
            valid_matrix_list.append(valid_matrix)
            valid_f1_list.append(valid_f1)
            
            
            if valid_f1 > best_f1:
                best_f1 = valid_f1
                torch.save(model.state_dict(), (model_dir+'/{}_{}').format(CFG['model'],fold))

            # EARLY STOPPING
            stop = early_stopping(valid_f1)
            if stop:
                print("stop called")   
                break
    
        end = time.time() - start
        time_ = str(datetime.timedelta(seconds=end)).split(".")[0]
        print("time :", time_)

        best_index = valid_f1_list.index(max(valid_f1_list))
        print(f'fold: {fold}, Best Epoch : {best_index}/ {len(valid_f1_list)}')
        
        print(f'fold: {fold}, Best Train Marco F1 : {train_f1_list[best_index]:.5f}')
        print(train_matrix_list[best_index])
        
        print(f'fold: {fold}, Best Valid Marco F1 : {valid_f1_list[best_index]:.5f}')
        print(valid_matrix_list[best_index])
        print('---------------------------------------------------------')
        
        if valid_f1_list[best_index] > best_fold:
            best_fold = valid_f1_list[best_index]
            top_fold = fold
    print(f'best_fold : {best_fold}\n Top fold : {top_fold}')

    del model, optimizer, train_loader, val_loader, scaler
    torch.cuda.empty_cache()


Model: efficientnet_b3
Training start with fold: 0 epoch: 200 

All training process
Fold: 0
Epoch 0/199


  0%|          | 0/53 [00:00<?, ?it/s]

  0%|          | 0/53 [00:00<?, ?it/s]